In [1]:
import pandas as pd
import numpy as np
import glob
import os
import plotly.express as px
from shapely.geometry.polygon import Polygon
import geopandas as gpd
from plotly.offline import plot
import plotly.graph_objects as go
import shapely
from shapely import wkt
import pandas as pd
import json
from dateutil import relativedelta
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import nbformat

filepath = '/Users/kmak/Desktop/EC.719/Building Energy Visualization/net-zero-dlab/Data files'


	
init_notebook_mode(connected=True)

/var/folders/09/m5s9tp9d3jv_16bdmpjm79rw0000gn/T/ipykernel_61677/3024724528.py:16: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

/var/folders/09/m5s9tp9d3jv_16bdmpjm79rw0000gn/T/ipykernel_61677/3024724528.py:17: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [2]:
energy = pd.read_csv(filepath + '/dash.csv') # read csv file
energy['date'] = pd.to_datetime(energy['date'], infer_datetime_format=True) # convert type of date column from string to datetime
# energy['geometry'] = gpd.GeoSeries.from_wkt(energy['geometry']) # convert type of geometry column

In [3]:
energy = energy.dropna(subset=['geometry'], axis=0, inplace=False) # remove buildings without a geometry

In [4]:
# remove all rows for energy types not Chilled Water, Electricity, or Steam
energy = energy[ (energy['energy_type'] == 'Chilled Water') | 
                (energy['energy_type'] == 'Electricity') | 
               (energy['energy_type'] == 'Steam')]

In [5]:
def df_to_geojson(df, properties, geometry='geometry'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Polygon',
                               'coordinates':[]}}

        polygon = row['geometry']
        feature['geometry']['coordinates'] = [np.asarray(polygon.exterior.coords).tolist()]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [6]:
buildingFootprints = pd.read_csv(filepath + '/building-footprint.csv')
buildingFootprints['geometry'] = gpd.GeoSeries.from_wkt(buildingFootprints['geometry'])

properties=['building']
geojson = df_to_geojson(buildingFootprints, properties)

In [7]:
startDate = energy['date'].min()
def monthsSince(date):
    diff = relativedelta.relativedelta(pd.to_datetime(date), pd.to_datetime(startDate)) # find time difference from initial measurement date
    return diff.months + diff.years * 12 # convert to months

def dateMark(date):
    return str(date)[5:7] + '/' + str(date)[2:4] # convert 2010-02-01 00:00:00 to 02/10
    

In [8]:
def markDisplay(i):
    if i % 4 == 0:
        return 'block'
    return 'none'

def colorDisplay(i):
    if i % 12 == 0:
        return 'blue'

In [9]:
def zmax(energyFormat):
    if energyFormat == 'MMBTU':
        return 6000
    if energyFormat == 'MMBTU_per_area':
        return 0.02
    if energyFormat == 'MMBTU_bloom':
        return 1

In [10]:
app = JupyterDash(__name__)

df = energy

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['energy_type'].unique(),
                'Chilled Water',
                id='energy-type'
            ),
            dcc.RadioItems(
                ['MMBTU', 'MMBTU_per_area', 'MMBTU_bloom'],
                'MMBTU',
                id='energy-format',
                inline=True
            )
        ], style={'width': '48%', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='choropleth-mapbox'),

    dcc.Slider(
        monthsSince(df['date'].min()),
        monthsSince(df['date'].max()),
        step=None,
        id='date--slider',
        value=monthsSince(df['date'].min()),
        marks={
            monthsSince(date): {'label': dateMark(date), 
                                'style': 
                                    {'display': markDisplay(i),
                                     'transform': 'rotate(-45deg)',
                                    'color': colorDisplay(i)}} 
            for i, date in enumerate(df['date'].unique())
        },
        updatemode='drag'
    )
])


@app.callback(
    Output('choropleth-mapbox', 'figure'),
    Input('energy-type', 'value'),
    Input('energy-format', 'value'),
    Input('date--slider', 'value'))
def update_graph(energy_type_name, energy_format_name, date_value):
    datedf = energy.copy()
    datedf['date'] = datedf['date'].apply(monthsSince)
    datedf = datedf[datedf['date'] == date_value] # extract rows of particular date
    dff = datedf[datedf['energy_type'] == energy_type_name] # extract rows of particular energy type
    
    fig = go.Figure()
    fig.add_trace(
            go.Choroplethmapbox(
                geojson=geojson,
                featureidkey = 'properties.building',
                locations = dff.building,
                z = dff[energy_format_name],
                colorscale='Reds',
                zmin = 0,
                zmax = zmax(energy_format_name),
                showscale=True,
                colorbar={"title": f'{energy_format_name}'},
         )
    )
    
    fig.update_layout(
          title=f'{energy_type_name} Usage',
          mapbox_style='carto-positron',
          autosize=True,
          mapbox_center={'lat':42.359, 'lon':-71.095},
          mapbox_zoom=14
        
    )

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 70, 'r': 0}, hovermode='closest')

    return fig


app.run_server(mode="inline")